# Yeast cells detection evaluation notebook

In [ ]:
import sys
sys.path.append('../')
#sys.path.append('%HOMEPATH%/detectron2') #insert local path to detecron2
import numpy as np

import yeastcells.clustering as clustering
import yeastcells.features as features
import yeastcells.data as data
import yeastcells.model as model
import yeastcells.visualize as visualize
import yeastcells.evaluation as evaluation

import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
cd ~

### Load data

In [ ]:
TS = 'TestSet7' #choose YIT test set
path = f'../../slow-data/yeastcells/data/cellstar_benchmark/{TS}'
filenames = data.load_data(path)

In [ ]:
image = data.read_images_cat(filenames)
np.shape(image)

### Load and run model on data for segmentation

In [ ]:
seg_path = f'{path}/GroundTruth_Segmentation.csv'
track_path = f'{path}/GroundTruth_Tracking.csv'
gt_s, gt_t = data.get_gt_yit(seg_path, track_path)

#set path to model_final.pth
model_path = '/var/tensorflow-logs/p253591/yeast-cell-detection-run-4'

In [ ]:
precision_s = []
recall_s = []
accuracy_s = []
F_s = []
precision_t = []
recall_t = []
accuracy_t = []
F_t = []

sts = [0.8,.81,.82,.83,.84,.85,.86,.87,.88,.89,.9,.91,.92,.93,.94,.95,.96,.97,.98,.99]

for i in range(len(sts)):
#load model
    predictor = model.load_model(model_path, seg_thresh=sts[i], device='cuda:0') #set GPU if available otherwise use 'cpu'

    output = [
      {'instances': predictor(frame)['instances'].to('cpu')}
      for frame in image
    ]

    labels, coordinates = clustering.cluster_cells(output, dmax=5, min_samples=3, eps=0.6, progress=False)

    pred_s, pred_t, _ = data.get_pred(output, labels, coordinates)

    pred_s_, results_s = evaluation.get_seg_performance(pred_s, gt_s, output, pipeline='maskrcnn') 
    pred_t_, results_t = evaluation.get_track_performance(pred_t, gt_t, output, pipeline='maskrcnn')

    metrics_s = evaluation.calculate_metrics(results_s, pred_s_, gt_s)
    metrics_t = evaluation.calculate_metrics(results_t, pred_t_, gt_t)
    
    precision_s.append(metrics_s['Precision'])
    recall_s.append(metrics_s['Recall'])
    accuracy_s.append(metrics_s['Accuracy'])
    F_s.append(metrics_s['F1-score'])
    precision_t.append(metrics_t['Precision'])
    recall_t.append(metrics_t['Recall'])
    accuracy_t.append(metrics_t['Accuracy'])
    F_t.append(metrics_t['F1-score'])

In [ ]:
plt.figure()
plt.title(f'Segmentation performance Mask RCNN {TS}')
plt.xlabel('Segmentation threshold')
plt.ylabel('Metrics')
plt.plot(sts, F_s, label='F1-score')
plt.plot(sts, accuracy_s, label='accuracy')
plt.plot(sts, precision_s, label='precision')
plt.plot(sts, recall_s, label='recall')
plt.ylim(0,1)
plt.legend()

In [ ]:
plt.figure()
plt.title(f'Tracking performance Mask RCNN {TS}')
plt.xlabel('Segmentation threshold')
plt.ylabel('Metrics')
plt.plot(sts, F_t, label='F1-score')
plt.plot(sts, accuracy_t, label='accuracy')
plt.plot(sts, precision_t, label='precision')
plt.plot(sts, recall_t, label='recall')
plt.ylim(0,1)
plt.legend()

In [ ]:
# rerun pipeline with optimized sts
predictor = model.load_model(model_path, seg_thresh=0.955, device='cuda:0') #set GPU if available otherwise use 'cpu'
output = [
      {'instances': predictor(frame)['instances'].to('cpu')}
      for frame in image
    ]
labels, coordinates = clustering.cluster_cells(output, dmax=5, min_samples=3, eps=0.6, progress=False)
pred_s, pred_t, _ = data.get_pred(output, labels, coordinates)
pred_s_, results_s = evaluation.get_seg_performance(pred_s, gt_s, output, pipeline='maskrcnn') 
pred_t_, results_t = evaluation.get_track_performance(pred_t, gt_t, output, pipeline='maskrcnn')
metrics_s = evaluation.calculate_metrics(results_s, pred_s_, gt_s)
metrics_t = evaluation.calculate_metrics(results_t, pred_t_, gt_t) 

print("%.4f" % metrics_s['Precision'])
print("%.4f" % metrics_s['Recall'])
print("%.4f" % metrics_s['Accuracy'])
print("%.4f" % metrics_s['F1-score'])
print("%.4f" % metrics_t['Precision'])
print("%.4f" % metrics_t['Recall'])
print("%.4f" % metrics_t['Accuracy'])
print("%.4f" % metrics_t['F1-score'])